# Computational Methods in Simulation (CMIS) Week 1
This notebook is intended to support your learning process. It contains exercises that help you to understand the material. 

This week is based on Slides 11 - follow the detailed steps below to learn about finite difference approximations and it's applications:

* Use a 4-by-4 grid for the small 2D Toy example from page 15 and apply the boundary conditions ∂u/∂x = 0  On all vertical boundaries and ∂u/∂y = 0 On all horizontal boundaries. Let f(x,y)=x+y and κ=2. 
* Draw the computational mesh
* Derive update formulas for the ghost nodes
* Derive update formulas for all domain nodes
* Explain how the approximation equations from the governing equations and boundary conditions are mapped into a matrix using index sets.
* Write code that assembles the full matrix system.
* Write code that assembles the corresponding matrix rows for the domain nodes.
* Write code that assembles the corresponding matrix rows for the ghost nodes
* Examine the fill pattern of the sub-block matrices from the Toy example. What can you say about them?
* Examine the eigenvalues of the full matrix system
* Speculate whether you can solve this linear system or not.

## Detailed Expected Learning Objectives of this Week

* Finite Difference Methods (FDM) Part 1 (Lecture slides number 11)
   * Explain the idea of using Finite Difference Approximations.
   * Derive the approximation for the 1st derivative using FD/BD/CD from the appropriate Taylor series expansions, derive the approximation of the 2nd derivative using CD from appropriate Taylor series expansion.
   * Define common terminology: A stencil, update scheme, computational mesh, matrix assembly etc.
    * Apply boundary conditions through FD/BD on boundary or ghost nodes.
    * Discuss pros and cons of an update scheme versus matrix assembly.

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

## The 2D TOY PDE problem

Solve 

$\nabla^2 u - k^2 u = f$,

given known $k > 0$ and $f(x,y)$

Use 

$\frac{\partial u}{\partial x} = 0$ 

for vertical boundaries. Use

$\frac{\partial u}{\partial y} = 0$ 

for horizontal boundaries

In [3]:
I, J = (6,6) # Size of computational domain including ghost boundary

# First we compute the x and y coordinates of all nodes in the computational
# mesh. Observe that we carefully ensure that the the domain boundary is
# half-way between the ghost nodes and the interior boundary nodes.
dx     = 1/4;
dy     = dx;
x = np.linspace(-dx/2,1+dx/2,6)
y = np.linspace(-dy/2,1+dy/2,6)

[X, Y]  = np.meshgrid(x,y)
U      = np.zeros((I,J),dtype=np.float64) 
k      = 2      # Just some arbitary value
f      = X+Y    # Just some arbitary value

fig = plt.figure()
ax = plt.subplot(111)
plt.hlines(y, x[0], x[-1])
plt.vlines(x, y[0], y[-1])
plt.plot([0, 1, 1, 0, 0],[0,0,1, 1,0], color='r')
ax.set_title("Computational Grid (black) Domain (red)")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Central diff approx are

We know from finite difference approximations that

$\frac{\partial^2 u}{\partial x^2} \approx  \frac{ u_{i+1,j} - 2 u_{i,j} + u_{i-1,j} }{\Delta x^2}$

$\frac{\partial^2 u}{\partial y^2} \approx  \frac{ u_{i,j+1} - 2 u_{i,j} + u_{i,j-1} }{\Delta y^2}$

Hence substituting into the Toy problem we obtain the approximation

$\nabla^2 u - k^2 u \approx c_{down} \, u_{i,j-1} +  c_{left}\, u_{i-1,j} + c_{mid} \,u_{i,j} + c_{right}\, u_{i+1,j} +  c_up \, u_{i,j+1}$

For all non-ghost nodes, $0<i<5$ and  $0<j<5$. Observe we assume that the lower left node in the domain has a zero-based index (0,0).

By method of equal coefficients we derive equations for the constants in the approximation given above.

In [4]:
# TODO - derive the update equations for the constants as in the example in the slides and define them here
c_left  = 
c_right = 
c_up    = 
c_down  = 
c_mid   = 

SyntaxError: invalid syntax (<ipython-input-4-41f4f7ddb817>, line 2)

Next we will write up all the stencil's simultaneously into a coefficient matrix

In the following cells, it is your job to
* Add code to assemble matrix A
* Add code to assemble right hand side term b
* Replace below with u = A\b

In [ ]:
D = [] # Index set of all domain nodes
G = [] # Index set of all ghost nodes

A = np.zeros((I*J,I*J), dtype=np.float64)  # The coefficient matrix of the full system
b = np.zeros((I*J,1), dtype=np.float64)    # The right hand side vector the full system

for j in range(1,I-1):
    for i in range(1,I-1):
        # TODO - Add code to assemble matrix A and right hand side term b using the hints below

        # Convert 2D index (i,j) into linear index space



        # Fill in the coefficient values into A and b
      
        
        

Before we handle the ghost nodes we will examine the assembly of the A-matrix

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
plt.spy(A)
ax.set_title("Fill Pattern of A before boundary conditions")
plt.show()

Next we investigate the eigenvalues of A. We know that we have 20 ghost nodes that are currently not accounted for. Further, We know that the Laplacian has negative eigenvalues. Hence, as we have 16 internal nodes, we must have 16 negative eigenvalues and 20 zero eigenvalues.

In 2D the Laplacian operator has 4 zero eigenvalues (any plane would be a solution, hence 4 numbers are free to choose). One may ask why should we not expect 24 zero-eigenvalues? That is because the 4 eigenmodes are shared by the ghost nodes on this grid. If we did the assembly without grid nodes then we would only find 4 zero eigenvalues.

In [ ]:
d,_ = np.linalg.eig(A)

fig = plt.figure()
ax = plt.subplot(111)
plt.plot(np.sort(d))
ax.set_title("Eigenvalues of A")
ax.set_xlabel("Index")
ax.set_ylabel("Value")
plt.show()

We can now continue adding stencils of the ghost nodes. First we handle the left and right boundary conditions at $x=0$ and $x=$. Here we find all $0<j<5$

$\left(\frac{\partial u}{\partial x}\right)_{\tfrac{1}{2}, j} \approx u_{1,j} - u_{0,j} = 0$

This means we have $u_{0,j} = u_{1,j}$. Similary for the right bouyndary

$\left(\frac{\partial u}{\partial x}\right)_{4+\tfrac{1}{2}, j} \approx u_{5,j} - u_{4,j} = 0$

From this we find  $u_{5,j} = u_{4,j}$

In the following cells, you need to add the stencils for the ghost nodes

In [ ]:
i = 0
for j in range(1,J-1):
    # TODO - Now we add the stencil for ghost node (i,j) to A and b
    ghost   = 
    domain  = 
    A[ghost,ghost] = 
    A[ghost,domain] = 
    b[ghost] = 
    G.append(ghost)
i = 5
for j in range(1,J-1):
    # TODO - Now we add the stencil for ghost node (i,j) to A and b
    ghost   = 
    domain  = 
    A[ghost,ghost] = 
    A[ghost,domain] = 
    b[ghost] = 
    G.append(ghost)    

In a similar fashion we can handle the top and bottom boundaries of our domain

Here we find all $0<i<5$

$\left(\frac{\partial u}{\partial y}\right)_{i, \tfrac{1}{2}} \approx u_{i,1} - u_{i,0} = 0$

Similary for the top bouyndary

$\left(\frac{\partial u}{\partial y}\right)_{i,4+\tfrac{1}{2}} \approx u_{i,5} - u_{i,4} = 0$

In [ ]:
j = 0
for i in range(1,I-1):
    # TODO - Now we add the stencil for ghost node (i,j) to A and b
    ghost = 
    domain = 
    A[ghost,ghost] =  
    A[ghost,domain] = 
    b[ghost] = 
    G.append(ghost)    
j = 5
for i in range(1,I-1):
    # TODO - Now we add the stencil for ghost node (i,j) to A and b
    ghost = 
    domain = 
    A[ghost,ghost] = 
    A[ghost,domain] = 
    b[ghost] = 
    G.append(ghost)    

Ghost corner cells have no influence on grid so we just set them to some values. If we do nothing then we will get into trouble as the A matrix will have a zero row for each corner ghost node. That will make A singular.

In [ ]:
i=0
j=0
ghost = I*j+i
A[ghost,ghost] = 1
b[ghost] = 0
G.append(ghost)

i=5
j=0
ghost = I*j+i
A[ghost,ghost] = 1
b[ghost] = 0
G.append(ghost)

i=0
j=5
ghost = I*j+i
A[ghost,ghost] = 1
b[ghost] = 0
G.append(ghost)

i=5
j=5
ghost = I*j+i
A[ghost,ghost] = 1
b[ghost] = 0
G.append(ghost)

Now we will verify if our boundary conditions was correctly assembled into the A matrix and that the A matrix is non-singular so we can solve our discrete PDE.

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
plt.spy(A)
ax.set_title("Fill Pattern of A after boundary conditions")
plt.show()

fig = plt.figure()
ax = plt.subplot(111)
plt.imshow(A)
ax.set_title("Matrix A shown as an image")
plt.colorbar()
plt.show()

In [ ]:
d,_ = np.linalg.eig(A)

fig = plt.figure()
ax = plt.subplot(111)
plt.plot(np.sort(d))
ax.set_title("Eigenvalues of A after boundary conditions")
ax.set_xlabel("Index")
ax.set_ylabel("Value")
plt.show()

Observe that the full matrix A is no longer a symmetric matrix after having added the boundary conditions. Hence, the eigenvalues are now complex numbers and no longer reals.

Finally we can now solve our discretized PDE problem. Observe that the solution will be given in the one-dimensional index space of the matrix system. Hence, to be able to visualize the solution we must convert back to the 2D index space.

When we show the solution we want to clip away the ghost boundary.

In [ ]:
u = np.linalg.solve(A,b)

U = np.zeros((I,J),dtype=np.float64)
for i in range(I):
    for j in range(J):
        idx = I*j+i
        U[i,j] = u[idx]


fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X[1:I-1,1:J-1],Y[1:I-1,1:J-1],U[1:I-1,1:J-1],cmap="magma", color='0.75', rstride=1, cstride=1)
ax.set_title("Solution of Toy Problem")
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("u")
plt.show()

## Another note on Eigenvalues:

While it is often useful as a guideline, the number of zero eigenvalues of our matrix A is not always related to the number of needed boundary conditions. The only thing a zero eigenvalue tells us, is that the discrete system is not well-posed and as such suggests that a modelling (we are missing some information) or discretization (we discretized our governing euqation wrongly) error was performed in earlier steps. Further, adding a boundary condition to a linear system is not the same as adding more rows to an underdetermined system. The reason is that boundary conditions modify both columns and rows when added. 

# Alternative solution: Shur reduction of the above 

Sofar we have solved the TOY problem by assembling a full system of equations and then solve the full system without exploiting any knowledge about domain nodes and ghost nodes. We will make a partitioning of the full system into small blocks and analyse these individually.

$
\begin{bmatrix}
A_{D,D} & A_{D,G} \\
A_{G,D} & A_{G,G} 
\end{bmatrix}
\begin{bmatrix}
u_{D} \\ u_{G}
\end{bmatrix}
=
\begin{bmatrix}
b_{D} \\ b_{G}
\end{bmatrix}
$

where $D$ is the index set of domain nodes and $G$ is the index set of the ghost nodes

In [ ]:
ADD = (A[D,:])[:,D]
ADG = (A[D,:])[:,G]
AGD = (A[G,:])[:,D]
AGG = (A[G,:])[:,G]
bD = b[D]
bG = b[G]

Next we will create the Schur reduction.

In our case AGG has either 1 or -1 on the diagonal. It means that it is its own inverse. That is

$A_{G,G}^{-1} = A_{G,G}$

This makes the Schur complement particular easy to compute for us. The Schur matrix has the benefit of being of lower dimension of A hence it is more efficient to solve the Schur system. Further, the Schur matrix is symmetric and definite.

In our case the Schur matrix will be negative definite due to the Laplace operator. However, if the entire Shur equation is multiplied by minus one then one can solve for $u$ using an iterative fast solver such as preconditioned conjugate gradient.

In [ ]:
S = ADD - np.dot(ADG,np.dot(AGG,AGD))
c = bD - np.dot(ADG,np.dot(AGG,bG))

fig = plt.figure()
ax = plt.subplot(111)
plt.spy(S)
ax.set_title("Fill Pattern of Schur Matrix")
plt.show()

fig = plt.figure()
ax = plt.subplot(111)
plt.imshow(S)
ax.set_title("Schur Matrix shown as an image")
plt.colorbar()
plt.show()

d,_ = np.linalg.eig(S)

fig = plt.figure()
ax = plt.subplot(111)
plt.plot(np.sort(d))
ax.set_title("Eigenvalues of Schur Matrix")
ax.set_xlabel("Index")
ax.set_ylabel("Value")
plt.show()

Finally, we will compute the solution for $u$ using the Schur complement and compare it against the solution of $u$ computed with the full system. It is interesting to know if this error is close the machine epsilon.

In [ ]:
uD = np.linalg.solve(S,c)

error = np.linalg.norm(uD-u[D]) / np.linalg.norm(u[D])
print('The relative error between Schur complement solution and full system solution is ', error)

print(np.finfo(dtype=np.float64))

That is all folks!